In [ ]:
"""
This code cell loops over the 20 images contained in women_clothing_subset, it combines each image with a prompt that tells claude-3.5-sonnet-v1 
what do we want exactly in order to analyze the image professionaly

"""
import boto3
import json
import base64
import os
import csv
from botocore.exceptions import ClientError

# AWS Bedrock client
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Claude model that supports images
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Fashion designer prompt
fashion_prompt = (
    "You are a professional high-end fashion designer with expertise in luxury garment analysis. "
    "You are reviewing a product photo from a premium fashion catalog for archival documentation. "
    "\n\n"
    "Analyze and describe the garment with expert precision, including:\n"
    "• Precise dimensions and proportions (length, fit, cut, silhouette)\n"
    "• Fabric composition, texture, and weight\n" 
    "• Construction details (stitching, seaming, finishing)\n"
    "• Color palette and tonal variations\n"
    "• Style influences and design references\n"
    "• Notable design features, embellishments, or hardware\n"
    "• Seasonal appropriateness and styling versatility\n"
    "\n"
    "Write in a concise yet sophisticated tone appropriate for a high-end fashion lookbook "
    "or design archive. Focus on technical accuracy and industry-standard terminology."
)

# Path to images
image_dir = "./women_clothing_subset"
output_csv = "gucci_captions.csv"

# Prepare CSV file with header
with open(output_csv, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["image_name", "caption"])

    # Loop through all image files
    for filename in os.listdir(image_dir):
        if not filename.lower().endswith((".jpg", ".jpeg", ".png", ".webp")):
            continue

        image_path = os.path.join(image_dir, filename)
        print(f"Processing: {filename}")

        # Detect correct media type
        ext = filename.lower().split(".")[-1]
        media_type = f"image/{'jpeg' if ext in ['jpg', 'jpeg'] else ext}"

        # Read and encode image
        with open(image_path, "rb") as img_file:
            image_base64 = base64.b64encode(img_file.read()).decode("utf-8")

        # Prepare the request
        native_request = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 512,
            "temperature": 0.3,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "source": {"type": "base64", "media_type": media_type, "data": image_base64}},
                        {"type": "text", "text": fashion_prompt}
                    ],
                }
            ],
        }

        try:
            # Invoke Claude
            response = client.invoke_model(
                modelId=model_id,
                body=json.dumps(native_request)
            )

            model_response = json.loads(response["body"].read())
            caption = model_response["content"][0]["text"]

            # Save to CSV
            caption = caption.replace("\n", " ")
            writer.writerow([filename, caption])

        except (ClientError, Exception) as e:
            print(f"ERROR processing {filename}: {e}")
            writer.writerow([filename, f"ERROR: {e}"])

print(f"\n✅ Captions saved to {output_csv}")

Processing: page1_High-rise_wide-leg_jeans_46.jpg
Processing: page1_Horsebit_wool_and_cashmere_sweater_43.jpg
Processing: page1_Hypnotic_floral_lace_midi_dress_48.jpg
Processing: page1_Hypnotic_floral_linen_and_silk_minidress_25.jpg
Processing: page1_Hypnotic_floral_linen_pencil_skirt_24.jpg
Processing: page1_Hypnotic_floral_ramie_blouse_23.jpg
Processing: page1_Hypnotic_linen_and_silk_organza_midi_dress_33.jpg
Processing: page1_Hypnotic_ruffled_top_51.jpg
Processing: page1_Iverness_knitted_cashmere_top_13.jpg
Processing: page1_Margan_cotton_poplin_shirt_15.jpg
Processing: page1_Ment_Bengal_stripe_cotton_poplin_shirt_3.jpg
Processing: page1_Ness_denim_jacket_5.jpg
Processing: page1_Off-shoulder_pleated_bustier_gown_17.jpg
Processing: page1_Printed_blouse_37.jpg
Processing: page1_Printed_silk_maxi_skirt_32.jpg
Processing: page1_Printed_wide-leg_pants_38.jpg
Processing: page1_Ribbed-knit_alpaca_and_yak_wool_cardigan_60.jpg
Processing: page1_Single-breasted_wool_coat_45.jpg
Processing: pa